In [224]:
import pandas as pd
import gzip
import json
import pickle
import zipfile
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import numpy as np
import matplotlib.pyplot as plt

import gensim
from gensim import corpora
from gensim.models import LdaModel
from gensim.models import TfidfModel
from sklearn.model_selection import train_test_split

In [225]:
# LDA (Latent Dirichlet Allocation), NMF (Non-Negative Matrix Factorization), and BERTopic 

In [226]:
file = 'preprocessed_reviews.pkl'

# Read the pickled file directly into a DataFrame
reviews = pd.read_pickle(file)
reviews.head()

,reviewerName,reviewText,summary,preprocessed_reviewText,tokens,review_lengths
0,Michelle W,The materials arrived early and were in excell...,Material Great,material arrive early excellent condition howe...,"[material, arrive, early, excellent, condition...",13
1,Rosalind White Ames,I am really enjoying this book with the worksh...,Health,really enjoy book worksheet make review goal m...,"[really, enjoy, book, worksheet, make, review,...",12
2,Allan R. Baker,"IF YOU ARE TAKING THIS CLASS DON""T WASTE YOUR ...",ARE YOU KIDING ME?,take class do nt waste money call book book is...,"[take, class, do, nt, waste, money, call, book...",28
3,Lucy,This book was missing pages!!! Important pages...,missing pages!!,book miss page important page could answer tes...,"[book, miss, page, important, page, could, ans...",11
4,Albert V.,I have used LearnSmart and can officially say ...,Best study product out there!,use learnsmart officially say amazing study to...,"[use, learnsmart, officially, say, amazing, st...",50


In [256]:
# random selection of 10000 reviews
reviews_10000 = reviews.sample(n=10000, random_state=1)

Preparing TF IDF 

In [257]:
# create a list of lists containing words for each review text (preprocessed)
#docs = reviews['preprocessed_reviewText'].tolist()
docs = reviews_10000['preprocessed_reviewText'].tolist()

# convert to a list of lists
docs = [doc.split() for doc in docs]

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(docs)

In [258]:
# Create Corpus
corpus = [dictionary.doc2bow(text) for text in docs]

In [259]:
# Apply TF-IDF Transformation
tfidf = TfidfModel(corpus)
tfidf_corpus = tfidf[corpus]

In [260]:
# Split the TF-IDF transformed corpus and texts into training and testing sets  only for lda_model_8020.sav
train_corpus, test_corpus, train_texts, test_texts = train_test_split(tfidf_corpus, docs, test_size=0.2, random_state=42)

LDA Model

In [261]:
# Train LDA Model using TF-IDF on the training set
lda_model2 = LdaModel(corpus=tfidf_corpus, id2word=dictionary, num_topics=5, passes=20)

# 71 min for lda_model 80/20 split, 5 topics, 50 passes

In [262]:
# Save the trained model to disk
pickle.dump(lda_model2, open('lda_model_10kR.sav', 'wb'))

In [263]:
# Load the saved model from disk
lda_model2 = pickle.load(open('lda_model_10kR.sav', 'rb'))

In [264]:
# Print topics
print("lda_model_10kR.sav")

topics = lda_model2.print_topics(num_words=5)
for topic in topics:
    print(topic)


lda_model_10kR.sav
(0, '0.017*"tax" + 0.010*"easy" + 0.010*"taxis" + 0.009*"year" + 0.009*"turbo"')
(1, '0.029*"great" + 0.028*"good" + 0.016*"work" + 0.011*"ok" + 0.008*"advertise"')
(2, '0.011*"love" + 0.002*"bible" + 0.001*"nancy" + 0.001*"_" + 0.001*"niece"')
(3, '0.004*"payroll" + 0.003*"expectation" + 0.003*"meet" + 0.002*"chrome" + 0.002*"cell"')
(4, '0.005*"use" + 0.005*"work" + 0.005*"product" + 0.005*"program" + 0.005*"software"')


In [265]:
# Load the saved model from disk
lda_model = pickle.load(open('lda_model_8020.sav', 'rb'))

In [266]:
# Print topics
print("lda_model_8020.sav")

topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

lda_model_8020.sav
(0, '0.005*"window" + 0.005*"use" + 0.005*"program" + 0.004*"work" + 0.004*"software"')
(1, '0.037*"game" + 0.013*"play" + 0.012*"kid" + 0.012*"fun" + 0.011*"map"')
(2, '0.029*"ok" + 0.025*"advertise" + 0.021*"awesome" + 0.020*"describe" + 0.016*"thank"')
(3, '0.010*"year" + 0.008*"use" + 0.007*"tax" + 0.006*"quicken" + 0.006*"product"')
(4, '0.083*"great" + 0.054*"good" + 0.048*"work" + 0.031*"love" + 0.027*"product"')


In [267]:
from gensim.models import CoherenceModel

# Lower perplexity and higher coherence scores indicate better model performance.

# Calculate Perplexity
#print('\nPerplexity: ', lda_model.log_perplexity(test_corpus))
print('\nPerplexity: ', lda_model2.log_perplexity(tfidf_corpus))

# Calculate Coherence Score
# coherence_model_lda = CoherenceModel(model=lda_model, texts=test_texts, dictionary=dictionary, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('\nCoherence Score: ', coherence_lda)

coherence_model_lda2 = CoherenceModel(model=lda_model2, texts=docs, dictionary=dictionary, coherence='c_v')
coherence_lda2 = coherence_model_lda2.get_coherence()
print('\nCoherence Score2: ', coherence_lda2)


# lda_model_80/20  Perplexity:  -9.585373895318991 , Coherence Score:  0.35134297496128297


Perplexity:  -9.039971818437143

Coherence Score2:  0.5169026087556776


In [239]:
# Perplexity: The log_perplexity method computes the model’s perplexity on the test set. It’s a measure of how well the model predicts the holdout sample and 
# is often used for comparing different topic models. Lower perplexity values indicate better generalization performance.

# Coherence Score: The CoherenceModel class from Gensim is used to compute the coherence score of the LDA model. A higher coherence score indicates that the words
# comprising the topics are more semantically related, making the topics more interpretable.

In [268]:
# Assign dominant topic to each review
dominant_topics = []
for doc_bow in corpus:
    topic_probs = lda_model.get_document_topics(doc_bow)
    dominant_topic = sorted(topic_probs, key=lambda x: x[1], reverse=True)[0][0]
    dominant_topics.append(dominant_topic)


In [269]:
print("lda_model_8020.sav")

# count how many diferent topics there are:
unique, counts = np.unique(dominant_topics, return_counts=True)

# create table with topics and counts
topics_counts = pd.DataFrame({'topic':unique, 'count':counts})
topics_counts

lda_model_8020.sav


,topic,count
0,0,9305
1,1,12
2,2,77
3,3,594
4,4,12


In [270]:
# Assign dominant topic to each review
dominant_topics2 = []
for doc_bow in corpus:
    topic_probs = lda_model2.get_document_topics(doc_bow)
    dominant_topic2 = sorted(topic_probs, key=lambda x: x[1], reverse=True)[0][0]
    dominant_topics2.append(dominant_topic2)

In [271]:
print("lda_model_10kR.sav")

# count how many diferent topics there are:
unique, counts = np.unique(dominant_topics2, return_counts=True)

# create table with topics and counts
topics_counts2 = pd.DataFrame({'topic':unique, 'count':counts})
topics_counts2

lda_model_10kR.sav


,topic,count
0,0,634
1,1,412
2,2,67
3,3,20
4,4,8867


In [283]:
# add dominant topic to dataframe
reviews_10000['Topic_LDA_model_8020'] = dominant_topics
reviews_10000['Topic_LDA_model_10kR'] = dominant_topics2

NMF

In [245]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

In [274]:
docs2 = reviews_10000['preprocessed_reviewText'].tolist()

In [275]:
# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=2000)
X = vectorizer.fit_transform(docs2)

In [276]:
num_topics = 5  

nmf = NMF(n_components=num_topics, random_state=42)
W = nmf.fit_transform(X)
H = nmf.components_

In [277]:
feature_names = vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(H):
    top_features = [feature_names[i] for i in topic.argsort()[-5:][::-1]]
    print(f"Topic {topic_idx}: {', '.join(top_features)}")

Topic 0: software, get, program, version, computer
Topic 1: great, product, price, item, deal
Topic 2: good, product, price, far, software
Topic 3: work, well, window, fine, expect
Topic 4: use, easy, year, tax, love


In [278]:
# asign topic to each review
reviews_10000['Topic_NMF'] = W.argmax(axis=1)

In [284]:
reviews_10000.head(10)

,reviewerName,reviewText,summary,preprocessed_reviewText,tokens,review_lengths,Topic_NMF,Topic_LDA_model_8020,Topic_LDA_model_10kR
417709,steve hayes,Simply does not work. I have a Panasonic camco...,Don't do it,simply work panasonic camcorder think would co...,"[simply, work, panasonic, camcorder, think, wo...",31,0,0,4
244812,Lori Evans,I ordered the cyberlink photo software & recei...,I ordered the cyberlink photo software & recei...,order cyberlink photo software receive can not...,"[order, cyberlink, photo, software, receive, c...",13,0,0,4
144600,Brncs,Had to upgrade for bank downloads to work. Wou...,Not bad,upgrade bank download work would prefer job re...,"[upgrade, bank, download, work, would, prefer,...",11,3,3,4
285232,Geoman,Do not buy this. I have been buying Turbo tax...,bait-and-switch,buy buying turbo tax last year never unlike ye...,"[buy, buying, turbo, tax, last, year, never, u...",19,4,0,4
443181,Willie Broughton,Users were very please with the Word perfect S...,Four Stars,user please word perfect suite,"[user, please, word, perfect, suite]",5,0,0,4
248874,Amazon Customer,SIDS not work as instructed,Two Stars,sid work instruct,"[sid, work, instruct]",3,3,3,1
115195,I love it,"<a data-hook=""product-link-linked"" class=""a-li...",Kaspersky Anti-Virus,datahookproductlinklinked classalinknormal hre...,"[datahookproductlinklinked, classalinknormal, ...",10,1,0,4
281814,Shirley Dorris,"Complete, fast, easy to use",easy to,complete fast easy use,"[complete, fast, easy, use]",4,4,0,0
301120,Dennis T Flick,Did the job well,Five Stars,job well,"[job, well]",2,3,3,1
395121,Krista,None of the games can be used by just tapping ...,Cute Games for kids who can use a mouse,none game use tap key require use mouse child ...,"[none, game, use, tap, key, require, use, mous...",21,4,0,4


In [252]:
reviews_10000["reviewText"].iloc[4]


'Users were very please with the Word perfect Suite'